In [42]:
import pandas as pd

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Reds
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter

import numpy as np


##### 1.2 - Import CSV file to pandas DataFrame

In [43]:
df = pd.read_csv('coord_data.csv')
# df = df[df['County'].isin(["Delaware, NY","Sullivan, NY", "Ulster, NY", "Orange, NY", "Rockland, NY", "Broome, NY", "Chenango, NY", "Otsego, NY", "Schoharie, NY", "Greene, NY"])]

display(df.head(60))

,County,New Positives,All Positives,New Tests,All Tests,% Positive,co-ordinates,latitude,longitude,geometry
0,"Albany, NY",163,7555,3138,287221,5.194391,"(42.65172576904297, -73.75509643554688)",42.588240,-73.974009,POINT (-73.75509643554688 42.65172576904297)
1,"Allegany, NY",44,1350,342,44322,12.865497,"(42.08806228637695, -78.49125671386719)",42.247853,-78.026153,POINT (-78.49125671386719 42.08806228637695)
2,"Bronx, NY",545,70085,13299,1413577,4.098052,"(40.82615280151367, -73.9202651977539)",40.848711,-73.852939,POINT (-73.92026519775391 40.82615280151367)
3,"Broome, NY",68,6306,2076,243869,3.275530,"(42.52092361450195, -74.30384826660156)",42.161977,-75.830283,POINT (-74.30384826660156 42.52092361450195)
4,"Cattaraugus, NY",90,1684,854,67431,10.538642,"(42.32959747314453, -78.86852264404297)",42.239099,-78.662332,POINT (-78.86852264404297 42.32959747314453)
5,"Cayuga, NY",97,1421,1226,77774,7.911909,"(42.91896057128906, -76.72624969482422)",43.008546,-76.574587,POINT (-76.72624969482422 42.91896057128906)
6,"Chautauqua, NY",121,2220,846,116271,14.302600,"(42.209285736083984, -79.46292877197266)",42.304216,-79.407595,POINT (-79.46292877197266 42.20928573608398)
7,"Chemung, NY",56,3651,1537,104484,3.643461,"(42.008331298828125, -76.62359619140625)",42.155280,-76.747179,POINT (-76.62359619140625 42.00833129882812)
8,"Chenango, NY",34,815,339,49176,10.029499,"(42.1686897277832, -75.87785339355469)",42.478024,-75.602241,POINT (-75.87785339355469 42.1686897277832)
9,"Clinton, NY",11,597,607,67431,1.812191,"(43.04859924316406, -75.37799072265625)",44.752712,-73.705643,POINT (-75.37799072265625 43.04859924316406)


##### 1.3 - Create Coordinate Transform Function

For this particular visualisation, we will need to convert our latitudes and longitudes into Mercator coordinates. To do this we will use a function ('borrowed' from my friend [Nadine Amersi-Belton](https://github.com/nadinezab/kc-house-prices-prediction)) to perform this transformation.

In [44]:
# Define function to switch from lat/long to mercator coordinates
def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

# Define coord as tuple (lat,long)
df['coordinates'] = list(zip(df['latitude'], df['longitude']))


# Obtain list of mercator coordinates
mercators = [x_coord(x, y) for x, y in df['coordinates'] ]


##### 1.4 - Add Mercator Coordinates to DataFrame

Now we will add those coordinates to our DataFrame.

In [45]:
# Create mercator column in our df
df['mercator'] = mercators

# Split that column out into two separate cols - mercator_x and mercator_y
df[['mercator_x', 'mercator_y']] = df['mercator'].apply(pd.Series)

# Drop 'geometry' column 
df = df.drop(columns=['geometry'])

# Examine our modified DataFrame
df.head()


,County,New Positives,All Positives,New Tests,All Tests,% Positive,co-ordinates,latitude,longitude,coordinates,mercator,mercator_x,mercator_y
0,"Albany, NY",163,7555,3138,287221,5.194391,"(42.65172576904297, -73.75509643554688)",42.588240,-73.974009,"(42.5882401, -73.9740095)","(-8234749.0694767805, 5249506.3306306675)",-8.234749e+06,5.249506e+06
1,"Allegany, NY",44,1350,342,44322,12.865497,"(42.08806228637695, -78.49125671386719)",42.247853,-78.026153,"(42.2478532, -78.0261531)","(-8685831.631650005, 5198179.261597844)",-8.685832e+06,5.198179e+06
2,"Bronx, NY",545,70085,13299,1413577,4.098052,"(40.82615280151367, -73.9202651977539)",40.848711,-73.852939,"(40.8487111, -73.852939)","(-8221271.563066695, 4990052.131028667)",-8.221272e+06,4.990052e+06
3,"Broome, NY",68,6306,2076,243869,3.275530,"(42.52092361450195, -74.30384826660156)",42.161977,-75.830283,"(42.1619773, -75.8302833)","(-8441388.523665678, 5185273.823934164)",-8.441389e+06,5.185274e+06
4,"Cattaraugus, NY",90,1684,854,67431,10.538642,"(42.32959747314453, -78.86852264404297)",42.239099,-78.662332,"(42.239098600000005, -78.66233199999999)","(-8756650.742851427, 5196862.816081127)",-8.756651e+06,5.196863e+06


### 2. Create Visualisation with Bokeh

##### 2.1 - Create Visualisation

The steps are described with comments below. Full explanations of each step can be found in the [associated article](https://towardsdatascience.com/creating-an-interactive-map-in-python-using-bokeh-and-pandas-f84414536a06).

This creates a [.html file](./output/avocado.html) which we can embed in dashboards or on [web pages](https://www.craigdoesdata.de/blog/avocadobokeh.html) as we please. The use of the 'show' function at the end also allows the visualisation to be displayed within our notebook.

In [49]:
# Create map with Bokeh

# Select tile set to use
chosentile = get_provider(Vendors.STAMEN_TONER)

myReds = ['#FFCCBB', '#FF6C5C', '#F85C4D', '#E84C3D', '#D83C2D']

# Choose palette
palette = myReds

# Reds[3].reverse()

# Tell Bokeh to use df as the source of the data
source = ColumnDataSource(data=df)

# Define color mapper - which column will define the colour of the data points
color_mapper = linear_cmap(field_name = '% Positive', palette = palette, low = df['% Positive'].min(), high = df['% Positive'].max())

# Set tooltips - these appear when we hover over a data point in our map, very nifty and very useful
nan_color = '#d9d9d9'
tooltips = [("Percemtage Positive","@% Positive"), ("County","@County")]


# Create figure
p = figure(title = 'Avocado Prices by region in the United States', x_axis_type="mercator", y_axis_type="mercator", 
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips, plot_width=1600, plot_height=800)

# Add map tile
p.add_tile(chosentile)

# Add points using mercator coordinates
p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=50, fill_alpha = 0.7)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

# Set color_bar location
p.add_layout(color_bar, 'right')

# Display in notebook
output_notebook()

# Save as HTML
output_file('avocado.html', title='Avocado Prices by region in the United States')

# Show map
show(p)

Loading BokehJS ...

---------------------

### 3. Conclusion
##### 3.1 - Conclusion

Now we have produced a working interactive map using Bokeh, which we can embed in our choice of online delivery system for our users / stakeholders. They are then free to examine the visualisation as they choose. 

